In [1]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [7]:
import re
import random
from fastai2.text.all import *
from fastai2.basics import *
from google_drive_downloader import GoogleDriveDownloader as gdd

In [4]:
bs=128
data_path = Config.config_path

lang = 'cs'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
path

Path('/home/jupyter/.fastai/cswiki')

In [16]:
# copy splitted wikipedia files
!rm -rf {path/'docs'} cswiki_tmp

gdd.download_file_from_google_drive(file_id='1JO8UJyL0Lqaq-V7uSQ0Nw4tqcCBRGQoR',
                                    dest_path='./cswiki.zip')

!unzip -q cswiki.zip -d "./cswiki_tmp"
!mv "./cswiki_data/root/.fastai/cswiki/docs" "{path}/docs/"

!rm -rf cswiki_tmp

In [13]:
# copy model and vocabulary
!rm -rf tmp
!mkdir tmp/

gdd.download_file_from_google_drive(file_id='1r4LWbdJACU0Cj9cCulwSmS2AeZyvBWr4',
                                    dest_path='tmp/spm.model')
gdd.download_file_from_google_drive(file_id='1FHb8jH2ZG3IB4vbVLZSinRdQsFVLuyVk',
                                    dest_path='tmp/spm.vocab')

!ls tmp/

spm.model  spm.vocab


In [18]:
dirpath = path/'docs'
filenames = os.listdir(dirpath)
N = len(filenames)

chunks_train = random.sample(filenames, int(N / 10 * 9))
chunks_valid = list(set(filenames) - set(chunks_train))

N, len(chunks_train), len(chunks_valid)


(118162, 106345, 11817)

In [20]:
!rm -rf data
!mkdir data
!mkdir data/train
!mkdir data/valid
!mkdir data/train/1
!mkdir data/valid/1

In [22]:
for chunk in chunks_train:
    shutil.copyfile(str(dirpath) + "/" + chunk, "data/train/1/" + chunk)
for chunk in chunks_valid:
    shutil.copyfile(str(dirpath) + "/" + chunk, "data/valid/1/" + chunk)

In [23]:
sp = SentencePieceTokenizer(
    lang='cs',
    sp_model=Path('tmp/spm.model'),
    max_vocab_sz=50000)

tkn = Tokenizer(sp)

In [24]:
tkn("Venku je panáček, sluníčko, už asi neuvidí.")

['▁xxbos',
 '▁xxmaj',
 '▁venku',
 '▁je',
 '▁pan',
 'áček',
 ',',
 '▁s',
 'lun',
 'íčko',
 ',',
 '▁už',
 '▁asi',
 '▁neu',
 'vid',
 'í',
 '.']